# Topic Models and their visualization
### Author: Eni Mustafaraj

This tutorial shows how to perform the topic modeling using the `sklearn` library. Additionally, it introduces the library `pyLDAvis`, a tool to visualize topic modeling that works with the results of both`sklearn` and `gensim`.
Here we will see it in action with `sklearn`. We will be using 20 newsgroups dataset as provided by scikit-learn.

**Table of Content**

1. [Prepare the notebook materials](#sec1)  
2. [Convert to document-term matrices](#sec2)  
3. [Fit Latent Dirichlet Allocation models](#sec3) 
4. [Visualizing the models with pyLDAvis](#sec4)
5. [YOUR TASK: Interpret the models](#sec5)

Create virtual environment:
python -m venv myenv

Activate:
Windows: myenv\Scripts\activate
MacOS: source myenv/bin/activate

Install packages:
pip install -r requirements.txt

 <a id="sec1"></a>
 ## 1. Prepare the notebook materials

From `sklearn` we'll be using one of its datasets, the feature extractors, and the LDA model.

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Now install `pyLDAvis`:

In [2]:
!pip install pyLDAvis

Import the needed packages and enable pyLDAvis to work inside the notebook:

In [5]:
# Ensure pyLDAvis is installed
%pip install pyLDAvis

# Import pyLDAvis and enable notebook visualization
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'pyLDAvis.sklearn'

### Load 20 newsgroups dataset

Below, the 20 newsgroups dataset available in `sklearn` is loaded. The headers, footers and quotes are removed, so that irrelevant words don't participate in the analysis.

In [8]:
newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))
docs_raw = newsgroups.data
print(len(docs_raw))

11314


<a id="sec2"></a>
## 2. Convert to document-term matrices

Next, the raw documents are converted into document-term matrix (dtm), initially as raw counts (with `CountVectorizer`) and then as TF-IDF values (with `TfidfVectorizer`).

We're creating two different vectorizers, to notice the difference that different feature representations make in the process of modeling.

In [9]:
# Create the TF vector represetnation, this only counts the terms in each document

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(11314, 9144)


Let's look at some of the features (i.e. words in the corpus):

In [10]:
tf_vectorizer.get_feature_names()[500:520]

['asks',
 'asleep',
 'aspect',
 'aspects',
 'ass',
 'assault',
 'assaults',
 'assemble',
 'assembled',
 'assembler',
 'assembly',
 'assert',
 'asserted',
 'asserting',
 'assertion',
 'assertions',
 'asserts',
 'assess',
 'assessment',
 'asshole']

To be able to compare the results of TF and TFIDF representation, we will use the same parameters.
This is why we will initialize the `TfidfVectorizer` with the parmeters of the `CountVectorizer`.

In [12]:
tf_vectorizer.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.5,
 'max_features': None,
 'min_df': 10,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': 'english',
 'strip_accents': 'unicode',
 'token_pattern': '\\b[a-zA-Z]{3,}\\b',
 'tokenizer': None,
 'vocabulary': None}

Initialize and prepare Document-Term-Matrix:

In [11]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())

dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

/Users/emustafa/opt/miniconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1799: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(11314, 9144)


<a id="sec3"></a>
## 3. Fit Latent Dirichlet Allocation models

We will fit two LDA models, each with 20 topics. Because the dataset is large, it takes a little bit to train the models.

In [13]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

lda_tf

LatentDirichletAllocation(n_components=20, random_state=0)

In [19]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)

Given that the models are probability distributions over topics and words, we will focus on their visualization to learn more about them.

<a id="sec4"></a>
## 4. Visualizing the models with pyLDAvis

There is a single method that we need to call, but, notice that this method has three parameters:

In [10]:
# the TF representation model

pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.080525  0.084967       1        1  10.698914
14     0.196207  0.062185       2        1  10.077951
0     -0.102869 -0.170439       3        1   9.104501
18     0.054788 -0.103025       4        1   7.036540
13     0.125707  0.045466       5        1   6.281324
8     -0.128318 -0.186213       6        1   5.411451
19    -0.012272 -0.152626       7        1   5.387479
12     0.094636  0.071910       8        1   5.377476
9     -0.124044  0.010609       9        1   4.547461
7     -0.030445 -0.011852      10        1   4.498512
5      0.142672  0.071664      11        1   4.342297
1     -0.061581 -0.055212      12        1   4.146615
15     0.069750 -0.060217      13        1   4.078443
2      0.110436 -0.003365      14        1   3.673889
11    -0.138638 -0.110874      15        1   3.391038
3     -0.176262 -0.024068      16        1   3.240042
17     0.115595  0.110588      17        1   2.977799
4      0.038802  0.044731      18        1   2.548118
10     0.079317  0.034016      19        1   2.216235
16    -0.334006  0.341754      20        1   0.963914, topic_info=            Term         Freq        Total Category  logprob  loglift
5016         max  4601.000000  4601.000000  Default  30.0000  30.0000
2653         edu  2438.000000  2438.000000  Default  29.0000  29.0000
3523         god  1945.000000  1945.000000  Default  28.0000  28.0000
4497         key  1211.000000  1211.000000  Default  27.0000  27.0000
7677       space  1250.000000  1250.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
8805     virtual    15.110336   134.053142  Topic20  -6.2580   2.4591
5140         mil    13.665991   113.450206  Topic20  -6.3585   2.5255
6642     reality    15.259194   189.037514  Topic20  -6.2482   2.1252
7221  scientific    15.126340   248.731356  Topic20  -6.2570   1.8420
2749         end    13.682255   852.822371  Topic20  -6.3573   0.5095

[1419 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
20       11  0.929034     absolutes
20       17  0.038710     absolutes
37       12  0.032754  accelerators
37       13  0.032754  accelerators
37       14  0.884362  accelerators
...     ...       ...           ...
9113     19  0.022005         years
9114     14  0.972649         yeast
9132      9  0.041934       yzerman
9132     18  0.922552       yzerman
9134      6  0.949301        zenith

[5975 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 15, 1, 19, 14, 9, 20, 13, 10, 8, 6, 2, 16, 3, 12, 4, 18, 5, 11, 17])

In [21]:
# The TFIDF representation model

pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.195576  0.043593       1        1  34.043427
6      0.120526  0.138568       2        1  20.066897
12     0.299475 -0.132850       3        1  12.540161
5      0.017137  0.188807       4        1   5.209774
10     0.104041  0.180372       5        1   4.859943
18     0.139450 -0.149233       6        1   3.561402
4     -0.120754 -0.015554       7        1   2.472662
7     -0.052732 -0.030381       8        1   2.044849
11    -0.029780 -0.073130       9        1   1.982630
13    -0.087406  0.005970      10        1   1.504950
9     -0.074859 -0.028866      11        1   1.408547
3     -0.025622 -0.053174      12        1   1.304860
14    -0.064001 -0.011279      13        1   1.287910
0     -0.072994 -0.015142      14        1   1.273234
1     -0.068393 -0.004103      15        1   1.262125
8     -0.064321 -0.018974      16        1   1.201762
17    -0.058346 -0.009243      17        1   1.042810
16    -0.054621 -0.008681      18        1   0.992643
19    -0.053633 -0.008068      19        1   0.970807
2     -0.048743  0.001367      20        1   0.968607, topic_info=        Term        Freq       Total Category  logprob  loglift
4497     key   52.000000   52.000000  Default  30.0000  30.0000
3523     god   82.000000   82.000000  Default  29.0000  29.0000
2653     edu   90.000000   90.000000  Default  28.0000  28.0000
8252  thanks  112.000000  112.000000  Default  27.0000  27.0000
1322    chip   42.000000   42.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
8267  theory    1.719096   16.841984  Topic20  -5.7826   2.3550
8906    wave    1.228214    8.026150  Topic20  -6.1188   2.7599
1002   bring    1.138414   14.067291  Topic20  -6.1947   2.1229
912     book    1.206459   37.538219  Topic20  -6.1367   1.1994
913    books    1.079081   20.675158  Topic20  -6.2483   1.6842

[1011 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
13       10  0.696130       abraham
24       19  0.444972     absurdity
49        2  0.821916      accident
51       14  0.746203  accidentally
88        2  0.249232           acs
...     ...       ...           ...
9131     18  0.288959           yup
9140      1  0.140220           zip
9140      6  0.070110           zip
9140      8  0.070110           zip
9140     12  0.701101           zip

[1994 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 7, 13, 6, 11, 19, 5, 8, 12, 14, 10, 4, 15, 1, 2, 9, 18, 17, 20, 3])

### [Optional] Using different MDS functions

With `sklearn` installed, other MDS functions, such as MMDS and TSNE can be used for plotting if the default PCoA is not satisfactory.

**Acronyms:**  
MDS = Multi-Dimensional Scaling   
PCoA = Principled Coordinates Analysis    
MMDS = Metric Multi-Dimensional Scaling  
TSNE = t-distributed Stochastic Neighbor Embedding   

That is, what changes is how the topics are visualized with respect to one-another in the 2D plane. Below, we show the same graphs, but this time with a different scaling technique.

In [20]:
# Notice the fourth parameter

pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.099949 -0.213273       1        1  10.698914
14    -0.061287 -0.120387       2        1  10.077951
0     -0.204806  0.220991       3        1   9.104501
18    -0.248929 -0.049583       4        1   7.036540
13     0.130947  0.032302       5        1   6.281324
8     -0.286411  0.172089       6        1   5.411451
19    -0.357969 -0.001664       7        1   5.387479
12    -0.200551 -0.291682       8        1   5.377476
9      0.002478 -0.374588       9        1   4.547461
7      0.092395 -0.255058      10        1   4.498512
5      0.108063 -0.102476      11        1   4.342297
1      0.057300  0.198581      12        1   4.146615
15    -0.138635  0.048003      13        1   4.078443
2     -0.024170  0.006722      14        1   3.673889
11    -0.111985  0.340480      15        1   3.391038
3      0.082585  0.355001      16        1   3.240042
17     0.244131 -0.230228      17        1   2.977799
4      0.291942 -0.110922      18        1   2.548118
10     0.272353  0.065793      19        1   2.216235
16     0.452499  0.309896      20        1   0.963914, topic_info=            Term         Freq        Total Category  logprob  loglift
5016         max  4601.000000  4601.000000  Default  30.0000  30.0000
2653         edu  2438.000000  2438.000000  Default  29.0000  29.0000
3523         god  1945.000000  1945.000000  Default  28.0000  28.0000
4497         key  1211.000000  1211.000000  Default  27.0000  27.0000
7677       space  1250.000000  1250.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
8805     virtual    15.110336   134.053142  Topic20  -6.2580   2.4591
5140         mil    13.665991   113.450206  Topic20  -6.3585   2.5255
6642     reality    15.259194   189.037514  Topic20  -6.2482   2.1252
7221  scientific    15.126340   248.731356  Topic20  -6.2570   1.8420
2749         end    13.682255   852.822371  Topic20  -6.3573   0.5095

[1419 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
20       11  0.929034     absolutes
20       17  0.038710     absolutes
37       12  0.032754  accelerators
37       13  0.032754  accelerators
37       14  0.884362  accelerators
...     ...       ...           ...
9113     19  0.022005         years
9114     14  0.972649         yeast
9132      9  0.041934       yzerman
9132     18  0.922552       yzerman
9134      6  0.949301        zenith

[5975 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 15, 1, 19, 14, 9, 20, 13, 10, 8, 6, 2, 16, 3, 12, 4, 18, 5, 11, 17])

In [22]:
# Different value for the 4th parameter

pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6      -10.878068   10.538728       1        1  10.698914
14     -60.884026  -75.591820       2        1  10.077951
0       -2.336119  -53.289074       3        1   9.104501
18     -61.385319  -12.914634       4        1   7.036540
13     -28.737743  126.296997       5        1   6.281324
8        1.066606 -123.552811       6        1   5.411451
19      45.140720   -7.947869       7        1   5.387479
12    -140.795975  -98.530548       8        1   5.377476
9       61.531681  -83.240562       9        1   4.547461
7      -72.563057 -140.396881      10        1   4.498512
5       15.984491   63.578186      11        1   4.342297
1     -160.851624   68.945145      12        1   4.146615
15      85.961830   51.922935      13        1   4.078443
2      -99.020836  109.936348      14        1   3.673889
11    -176.764130   -8.681684      15        1   3.391038
3     -117.189880  -37.420200      16        1   3.240042
17     108.829918  -22.946203      17        1   2.977799
4       47.588097  121.934814      18        1   2.548118
10    -106.467834   30.176889      19        1   2.216235
16     -50.613804   59.328842      20        1   0.963914, topic_info=            Term         Freq        Total Category  logprob  loglift
5016         max  4601.000000  4601.000000  Default  30.0000  30.0000
2653         edu  2438.000000  2438.000000  Default  29.0000  29.0000
3523         god  1945.000000  1945.000000  Default  28.0000  28.0000
4497         key  1211.000000  1211.000000  Default  27.0000  27.0000
7677       space  1250.000000  1250.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
8805     virtual    15.110336   134.053142  Topic20  -6.2580   2.4591
5140         mil    13.665991   113.450206  Topic20  -6.3585   2.5255
6642     reality    15.259194   189.037514  Topic20  -6.2482   2.1252
7221  scientific    15.126340   248.731356  Topic20  -6.2570   1.8420
2749         end    13.682255   852.822371  Topic20  -6.3573   0.5095

[1419 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
20       11  0.929034     absolutes
20       17  0.038710     absolutes
37       12  0.032754  accelerators
37       13  0.032754  accelerators
37       14  0.884362  accelerators
...     ...       ...           ...
9113     19  0.022005         years
9114     14  0.972649         yeast
9132      9  0.041934       yzerman
9132     18  0.922552       yzerman
9134      6  0.949301        zenith

[5975 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 15, 1, 19, 14, 9, 20, 13, 10, 8, 6, 2, 16, 3, 12, 4, 18, 5, 11, 17])

<a id="sec5"></a>
## 5. Your task: Interpret the models

First, watch the video on slide 23 about using pyLDAvis with the 20 Newsgroup dataset.  
If you need more context, read these other sources:

- [The 20 Newsgroup dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html)
- [Homepage for the 20 Newsgroup dataset](http://qwone.com/~jason/20Newsgroups/)

Then, play with the four visualizations above to get a better sense of the topics.

**Questions to answer below:**

1. Repeat the two final visualizations, for the TFIDF representation.
2. Which of the four visualizations seems more helpful to you for understanding the topics? Explain.
2. Can you map 